## Previo
- Agarrar los datos del tp anterior
- Bajarse el dataset de testing de Kaggle (no usado actualmente)

In [5]:
from __future__ import division
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.decomposition import IncrementalPCA

In [6]:
properati = pd.read_csv('properati_features.csv')
properati = properati.set_index(properati.columns[0])
properati.head(5)

,lat,lon,surface_total_in_m2,surface_covered_in_m2,rooms,description,departamento,ph,casa,Capital Federal,...,Villa Brown,Sourigues,Monte Chingolo,Fincas de Iraola,Cuartel V,El Rocío,Aldo Bonzi,Mariano Acosta,Los Pingüinos Country Club,Malvinas Argentinas
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,-34.610988,-58.363464,0.0,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,NaN,NaN,0.0,NaN,NaN,El departamento cuenta con un living-comedor a...,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,-34.589363,-58.412880,53.0,48.0,NaN,IMPECABLE TORRE COY III – DEPA...,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,NaN,NaN,39.0,39.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,NaN,NaN,51.0,51.0,NaN,LIVING COMEDOR CON PISOS DE PO...,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Inicializo el DF

In [ ]:
df = pd.read_csv('data_gba_total_features.csv')

def extra(row, list_of_words):
    if pd.isnull(row['title']):
        return False
    for word in list_of_words:
        if word in row['title']:
            return True
    return False

df['pileta'] = df.apply((lambda x: extra(x, ['pile'])), axis=1)
df['jardin'] = df.apply((lambda x: extra(x, ['patio', 'jardin'])), axis=1)
df['garage'] = df.apply((lambda x: extra(x, ['garage', 'cochera'])), axis=1)

df['casa'] = df['house']
df['departamento'] = df['apartment']
df['ph'] = df['PH']
#del df['extra']  # literalmente todos nulls
#del df['image_thumbnail']
#del df['properati_url']
del df['surface_in_m2']
df.sample(5)

In [102]:
df = df.dropna()
price = df['price_aprox_usd']
df = df[properati.columns]  # Consistencia con el set de predicción
df.sample(5)

,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,departamento,ph,casa,Capital Federal,...,Villa Brown,Sourigues,Monte Chingolo,Fincas de Iraola,Cuartel V,El Rocío,Aldo Bonzi,Mariano Acosta,Los Pingüinos Country Club,Malvinas Argentinas
219847,-34.541299,-58.463423,90.0,84.0,9.0,3.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
605877,-34.651700,-58.559943,37.0,37.0,2.0,2.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49082,-34.561423,-58.492063,47.0,39.0,1.0,2.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
617288,-34.607898,-58.564493,82.0,70.0,2.0,2.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
44514,-34.599574,-58.378334,43.0,43.0,10.0,3.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
for i in range(5, len(df.columns)):
    pca = IncrementalPCA(n_components=i)
    r = pca.fit_transform(df)
    r = pd.DataFrame(r)
    param_grid = {'max_features': range(1, len(r.columns))}
    x_train, x_test, y_train, y_test = train_test_split(r, price)

    reg = RandomForestRegressor()
    grid = GridSearchCV(reg, param_grid, cv=3)
    grid.fit(x_train, y_train)
    print(i, grid.best_score_)

(5, 0.96933573849461141)
(6, 0.95086716923047143)
(7, 0.9605222949995369)
(8, 0.95945786623890716)
(9, 0.95946947442587061)


A partir de los resultados anteriores vemos que cantidad de dimensiones es la que mejor score da.

Aca pruebo el mejor estimador encontrado por el grid search (el que usa los mejores hiperparametros encontrados) unas cuantas veces con sets random a ver cuanto da el error promedio

In [ ]:
errors = []
estimator = RandomForestRegressor(n_estimators=10)
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(r, price)
    estimator.fit(x_train, y_train)
    result = estimator.predict(x_test)
    errors.append(mean_squared_error(y_test, result))


In [ ]:
pd.DataFrame(errors).describe()